<a href="https://colab.research.google.com/github/SaulHuitzil/Boolean-Networks/blob/main/Boolean_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import math
import datetime
import numpy as np
from typing import List, Dict, Any
from tqdm import tqdm


In [ ]:
class Nodo:
    """
    Análogo a la clase Nodo en C#.
    """
    def __init__(self):
        self.Entradas: List[int] = []
        self.FB: Dict[str, str] = {}
        self.random = random.Random()

    def Clonar(self) -> 'Nodo':
        clon = Nodo()
        clon.Entradas = list(self.Entradas)
        clon.FB = dict(self.FB)
        return clon

    def CrearEntradasConPoisson(self, numNodos: int, lambda_: float):
        """
        Genera un número de entradas muestreando con Poisson(lambda_), asegurando al menos 1 entrada.
        """
        while True:
            numeroEntradas = self.PoissonSample(lambda_)
            numeroEntradas = min(numeroEntradas, numNodos - 1)
            if numeroEntradas >= 1:
                break
        # Escogemos aleatoriamente `numeroEntradas` de entre [0..numNodos-1]
        self.Entradas = random.sample(range(numNodos), numeroEntradas)

    def PoissonSample(self, lambda_: float) -> int:
        """
        Muestreo de Poisson(lambda_).
        """
        L = math.exp(-lambda_)
        p = 1.0
        k = 0
        while True:
            k += 1
            p *= self.random.random()
            if p <= L:
                break
        return k - 1

    def CrearEntradas(self, conectividad: int, numNodos: int):
        """
        Crea entradas tomando `conectividad` nodos únicos (aleatorios) de 0..numNodos-1.
        """
        self.Entradas = random.sample(range(numNodos), conectividad)

    def CrearFB(self):
        """
        Crea la tabla de verdad para las `Entradas`.
        """
        conectividad = len(self.Entradas)
        for i in range(int(math.pow(2, conectividad))):
            clave = bin(i)[2:].zfill(conectividad)
            self.FB[clave] = str(random.randint(0, 1))

    def AgregarEntrada(self, nuevaEntrada: int):
        """
        Agrega una entrada y actualiza la función booleana (FB).
        """
        self.Entradas.append(nuevaEntrada)
        oldFB = dict(self.FB)
        self.FB.clear()
        for key in oldFB.keys():
            # Al extender la configuración con un bit extra, la mitad de las nuevas
            # configuraciones mantiene el valor, la otra mitad será un nuevo valor aleatorio.
            key0 = key + '0'
            key1 = key + '1'
            self.FB[key0] = oldFB[key]
            self.FB[key1] = str(random.randint(0, 1))

    def QuitarEntrada(self, entrada: int):
        """
        Elimina la entrada y modifica la función FB.
        """
        indice = self.Entradas.index(entrada)
        self.Entradas.pop(indice)
        nuevoFB = {}
        for conf in self.FB.keys():
            # Solo conservamos aquellas configuraciones donde el bit 'indice' sea '0'.
            if conf[indice] == '0':
                # Removemos el bit en la posición 'indice'
                new_key = conf[:indice] + conf[indice+1:]
                nuevoFB[new_key] = self.FB[conf]
        self.FB = nuevoFB



In [ ]:
class Red:
    """
    Análogo a la clase Red en C#.
    """
    def __init__(self):
        self.Nodos: List[Nodo] = []
        self.Error: float = 0.0
        self.TasaMut: float = 0.01
        self.KObjetivo: float = 0.0
        self.SObjetivo: float = 0.0

        self.p_agregarConexion = 0.5
        self.p_encenderNodo = 0.5
        self.random = random.Random()

    def Clonar(self) -> 'Red':
        clon = Red()
        clon.TasaMut = self.TasaMut
        clon.KObjetivo = self.KObjetivo
        clon.Nodos = [n.Clonar() for n in self.Nodos]
        return clon

    def CrearRed(self, numNodos: int, conectividad: float):
        self.KObjetivo = conectividad
        self.SObjetivo = 2 * 0.5 * (1 - 0.5) * self.KObjetivo
        self.Nodos = []
        for _ in range(numNodos):
            nodo = Nodo()
            nodo.CrearEntradasConPoisson(numNodos, conectividad)
            nodo.CrearFB()
            self.Nodos.append(nodo)

    def Actualizar(self, estado0: str) -> str:
        estado1 = []
        for nodo in self.Nodos:
            configuracion = []
            for entrada in nodo.Entradas:
                configuracion.append(estado0[entrada])
            configuracion_str = ''.join(configuracion)
            estado1.append(nodo.FB[configuracion_str])
        return ''.join(estado1)


    def Conectividad(self) -> float:
        return float(sum(len(n.Entradas) for n in self.Nodos)) / len(self.Nodos)

    def P(self) -> float:
        """
        Fracción de salidas que son '1' en la tabla FB de todos los nodos.
        """
        bits = ''.join(''.join(nodo.FB.values()) for nodo in self.Nodos)
        return bits.count('1') / len(bits) if bits else 0.0

    def Sensitividad(self) -> float:
        k = self.Conectividad()
        p_ = self.P()
        return 2 * p_ * (1 - p_) * k

    def Mutar(self):
        """
        Recablea las entradas, tratando de alcanzar KObjetivo (mutación topológica).
        """
        for nodo in self.Nodos:
            if self.random.random() < self.TasaMut:
                if self.random.random() < 0.5 + (self.KObjetivo - self.Conectividad()) / self.KObjetivo:
                    # Agregar entrada
                    disponibles = [i for i in range(len(self.Nodos)) if i not in nodo.Entradas]
                    if len(disponibles) > 0:
                        nuevaEntrada = self.random.choice(disponibles)
                        nodo.AgregarEntrada(nuevaEntrada)
                else:
                    # Quitar entrada
                    if len(nodo.Entradas) > 1:
                        entrada = self.random.choice(nodo.Entradas)
                        nodo.QuitarEntrada(entrada)



1111001100
1000001010
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101
0000000110
1001101101